In [ ]:
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter
import matplotlib.image
import os
import sys
sys.path.append('/home/bekah/gPhoton2')


In [ ]:
### Way I used to stack dosemaps and filter out stars 

fuv = pd.read_csv("nuv_no_ngs.csv")
eclipse_list = fuv['eclipse'][0:]

cumulative_image = None
eclipse_counter = 0
file = 0
for e in eclipse_list:
    if eclipse_counter <= 5000:
        i = str(e).zfill(5)
        # for mounted backplanetest bucket
        file_path = f"/mnt/s3/e{i}-nd/e{i}-nd-b00-f3000-t00000-g_dose.fits.gz"
        if os.path.isfile(file_path):
            try:
                with fits.open(file_path) as hdul:
                    data = hdul[0].data
                    if data is None:
                        print("no data")
                    if cumulative_image is None:
                        cumulative_image = np.zeros_like(data, dtype=np.float64)
                    if data is not None:
                        print(f"adding eclipse {e}, spot {eclipse_counter}")
                        data[data > 3] = 0
                        cumulative_image += data.astype(np.float64)
                        eclipse_counter = eclipse_counter + 1
            except Exception as ex:
                print(f"probably a corrupt fits file, {ex}")
        else:
            print(f"dose map ({file_path}) does not exist")

hdu = fits.PrimaryHDU(cumulative_image)
hdul = fits.HDUList([hdu])
combo_filename = "/mnt/s3/nuv_5000stack_filtered3.fits"
hdul.writeto(combo_filename, overwrite=True)

In [ ]:
### Below is one way to make a stacked dosemap (flat!) and get rid of really bright stars etc to see background 

In [ ]:
# band 
b = 'f'

In [ ]:
# STEP ONE
# stack backplane stacks to get an "all dose" image 

def stack_fits_images(file_list, output_path):
    stacked_image_data = None
    for i, file_path in enumerate(file_list):
        with fits.open(file_path) as hdul:
            image_data = hdul[0].data
            if stacked_image_data is None:
                stacked_image_data = np.zeros_like(image_data, dtype=np.float64)
            stacked_image_data += image_data
    hdu = fits.PrimaryHDU(stacked_image_data)
    hdul = fits.HDUList([hdu])
    hdul.writeto(output_path, overwrite=True)
    print(f"Summed image saved to {output_path}")

# get file list 

directory = f'/media/bekah/BekahA/hotspot/200_stack_{b}uv/correct_stacks'

file_list = [f for f in os.listdir(directory) if f.endswith(('.fits'))]
file_paths = [os.path.join(directory, file_name) for file_name in file_list]

output_path = f'/media/bekah/BekahA/hotspot/200_stack_{b}uv/correct_stacks/all_{b}uv_summed_image.fits'
stack_fits_images(file_paths, output_path)

In [ ]:
# STEP TWO
# dividing stacked image data by the same data with a median filter to produce "filtered" image data
fts2 = fits.open(f'/media/bekah/BekahA/hotspot/200_stack_{b}uv/correct_stacks/all_{b}uv_summed_image.fits')

image = fts2[0].data

filtered_image = median_filter(image, size=10)
filtered = np.ma.divide(image, filtered_image).filled(0)  # Filled with 0 where b is zero
#mask = (image <= 10) | (filtered_image > median*20)
#binary_array = mask.astype(np.uint8)
hdu = fits.PrimaryHDU(filtered)
hdul = fits.HDUList([hdu])
hdul.writeto(f'/media/bekah/BekahA/hotspot/200_stack_{b}uv/correct_stacks/filtered_10_{b}.fits', overwrite=True)
print(f"image saved")

In [ ]:
# STEP THREE
# masking the normalized images created above in "filtered"

fts2 = fits.open(f'/media/bekah/BekahA/hotspot/200_stack_{b}uv/correct_stacks/filtered_10_{b}.fits')
image = fts2[0].data
mask = (image <= .75) | (image > 1.75)
binary_array = mask.astype(np.uint8)
hdu = fits.PrimaryHDU(binary_array)
hdul = fits.HDUList([hdu])
hdul.writeto(f'/media/bekah/BekahA/hotspot/200_stack_{b}uv/correct_stacks/filtered_10_mask_n.fits', overwrite=True)
print(f"image saved")